In [1]:
%load_ext autoreload
%autoreload 2

from importers import SpikeImporter

data_folder = "../data/"
file_name = "11_10_27U2a_2000s_2300s_20000Hz_Force_AllFibres"
file_ending = ".csv"

time_channel = "Time"
signal_channel = "1 Signal"
stimulus_channel = "32 DigMark"
ap_marker_channels = ["801 nw-1", "12 nw-1", "14 nw-1"]
force_channel = "3 Force"
# extra_stimuli_channel = "801 DigMark"

# load the animal file
importer = SpikeImporter(filepath = data_folder + file_name + file_ending, time_channel = time_channel, signal_channel = signal_channel)

print("Data loaded.")

# define the max. time so that two sequential waveform-values are belonging to the same AP
# the time is in seconds, so 0.005 equals 5ms
max_gap_time = 0.005
force_thresh = 0.5

# retrieve the events from the importer 
el_stimuli = importer.get_electrical_stimuli(regular_stimulus_channel = stimulus_channel)
# ex_stimuli = importer.get_extra_stimuli(extra_stimulus_channel = extra_stimuli_channel, regular_el_stimuli = el_stimuli)
mech_stimuli = importer.get_mechanical_stimuli(force_channel = force_channel, threshold = force_thresh, max_gap_time = 0.005)
actpots = importer.get_action_potentials(max_gap_time = max_gap_time, ap_marker_channels = ap_marker_channels, \
                                       el_stimuli = el_stimuli, mech_stimuli = mech_stimuli)

Data loaded.
List of eletrical stimuli created.
List of mechanical stimuli created.
Finished processing AP channel 1 out of 3
Finished processing AP channel 2 out of 3
Finished processing AP channel 3 out of 3
List of APs created.


In [6]:
''' 
This cell performs the clustering according to
- distance to previous stimulus
- normalized signal energy of the APs
'''
%autoreload 2
from fibre_tracking import DBSCANClustering

clustered_data_df = DBSCANClustering.perform_clustering(actpots, eps = 0.001, min_samples = 10)

print(clustered_data_df)

          Onset     Offset  Latency      Energy  Channel_Index  Cluster_Index
0       0.06615    0.06730  0.00970  335.889889              0              0
1       0.31640    0.31755  0.00970  284.552985              0              0
2       0.56660    0.56775  0.00970  331.157419              0              0
3       0.81685    0.81800  0.00975  234.416318              0              0
4       1.06700    1.06815  0.00965  270.243484              0              0
...         ...        ...      ...         ...            ...            ...
2770  299.04440  299.04700  0.22430   56.928723              2              3
2771  299.26135  299.26395  0.19105   52.291433              2             14
2772  299.34205  299.34465  0.02155   45.686410              2              1
2773  299.54000  299.54260  0.21950   46.606737              2              3
2774  299.78840  299.79100  0.21765   46.274020              2              3

[2775 rows x 6 columns]


In [9]:
%autoreload 2
from plotting import ClusterPlot2D

cl2dplot = ClusterPlot2D()
cl2dplot.plot(clustered_data_df = clustered_data_df)

In [4]:
%autoreload 2

from plotting import FallingLeafPlot
import itertools

analysis_start = 32
analysis_stop = 36

clusters_to_plot = [0, 1]
indices = [(True if lbl in clusters_to_plot else False) for lbl in labels]
track_aps = list(itertools.compress(actpots, indices))

# get the raw signal from the data
raw_signal_intervals = importer.get_raw_signal_split_by_stimuli(el_stimuli = el_stimuli, start_time = analysis_start, stop_time = analysis_stop, verbose = False)
max_signal = max([max(interval) for interval in raw_signal_intervals])

flplot = FallingLeafPlot()
flplot.plot(regular_stimuli = el_stimuli, action_potentials = track_aps, plot_raw_signal = True, plot_hlines = True, time_start = analysis_start, time_stop = analysis_stop, post_stimulus_timeframe = 0.02)
flplot.save_to_file(filename = "../results/falling_leaf_" + file_name + ".png")

NameError: name 'labels' is not defined